# Setup

In [1]:
import pickle
from tqdm.auto import tqdm as progressbar
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as c
import importlib
importlib.reload(c)
import common as c
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients
%matplotlib qt5

## Load calib

In [2]:
import pprint

# Calib params
with open("calib.pkl", "rb") as f:
    calib = pickle.load(f)
pprint.pprint(calib)
mtx = calib["mtx"]
dist = calib["dist"]
n = calib["n"]

# Aruco Board
board, aruco_dict = load_board("board.pkl")
print(board.getChessboardSize())

# Triang images
with open("output_triang.pkl", "rb") as f:
    res = pickle.load(f)
    
pprint.pprint(res)

images=[]
for data in progressbar(res):
    hdr = c.hdr(list(c.decode(d) for d in data["pictures"]), data["exposures"])
    hdr = cv2.undistort(hdr, mtx, dist)
    images.append((data["point"], hdr))

{'dist': array([[ 1.26203062e-01, -4.74487420e+00,  6.66135867e-04,
        -1.22549516e-02,  4.96963351e+01]]),
 'mtx': array([[3.03057567e+03, 0.00000000e+00, 4.55625243e+02],
       [0.00000000e+00, 8.27594165e+03, 6.41064611e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'n': array([-6.23611776e-04, -3.59899787e-02,  2.32149048e-03, -9.99349260e-01])}
(8, 8)
[{'exposures': [50, 100, 500, 1000],
  'pictures': [<Response [200]>,
               <Response [200]>,
               <Response [200]>,
               <Response [200]>],
  'point': (125.0, 90.0, 140.0)},
 {'exposures': [50, 100, 500, 1000],
  'pictures': [<Response [200]>,
               <Response [200]>,
               <Response [200]>,
               <Response [200]>],
  'point': (125.0, 90.71428571428571, 140.0)},
 {'exposures': [50, 100, 500, 1000],
  'pictures': [<Response [200]>,
               <Response [200]>,
               <Response [200]>,
               <Response [200]>],
  'point': (125.0, 91.428

100%|██████████| 50/50 [00:25<00:00,  1.94it/s]


In [56]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(images[0][1], animated=True)
    pointsplot, = plt.plot(range(images[0][1].shape[0]), "r", animated=True)
    return pointsplot, imgplot

def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    point, image = images[i]
    centroids = c.column_centroids(c.red_contrast(image), mask=c.red_contrast(image)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    # Update anim
    imgplot.set_data(image)
    pointsplot.set_data(p_centroids[0], p_centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=200, interval=20, blit=True)

In [3]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

In [62]:
W = np.eye(4)
arucoParams = aruco.DetectorParameters_create()
Xlist = []
for (point, image) in progressbar(images):


    centroids = c.column_centroids(c.red_contrast(image), mask=c.red_contrast(image)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T
    p_centroids = p_centroids[:,::10]

    pts_3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
    # print(pts_3d.shape)

    img_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    corners, ids, rejected = aruco.detectMarkers(img_gray, aruco_dict, parameters=arucoParams)
    resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
        markerCorners=corners, markerIds=ids, image=img_gray, board=board
    )
    ret, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, board, mtx, None, None, None)
    rot, jac = cv2.Rodrigues(rvec)
    W[:3, -1] = tvec.flatten()
    W[:3,:3] = rot

    Xlist.append(np.linalg.inv(W)@pts_3d)
X = np.hstack(Xlist)

100%|██████████| 50/50 [00:04<00:00, 10.48it/s]


In [73]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
region = X[2] > 0
ax.scatter(X[0][region],X[1][region], X[2][region], color="r")

d:\pedro\TCC\ender-laser-scanner\myenv\lib\site-packages\ipykernel\eventloops.py:105: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  app.exec_()


In [77]:
region = X[2] != 9999
plt.scatter(X[1][region], X[2][region])